Submission probabilities tweaks

In [22]:
import numpy as np
import pandas as pd

In [32]:
df = pd.read_csv("../results/submission_merge__mean_cv=6_squeezenet_fold=0_mixed_cnn_2017-06-10-11-50.csv")

In [31]:
df.head()

,image_name,Type_1,Type_2,Type_3
0,0.jpg,0.235615,0.514373,0.250013
1,1.jpg,0.409603,0.412734,0.177662
2,10.jpg,0.251292,0.632521,0.116187
3,100.jpg,0.231874,0.633217,0.134909
4,101.jpg,0.452883,0.445858,0.101259


In [45]:
def foo(probas):
    imax = np.argmax(probas.values)
    indices = range(3)
    indices.remove(imax)
    probas[imax] = 0.95
#     probas[indices] = 0.025
    return probas

In [46]:
cols = ['Type_%i' % i for i in range(1, 4)]
df[cols] = df.loc[:, cols].apply(foo, axis=1)

In [47]:
import os
from datetime import datetime

info = 'merge__mean_cv=6_squeezenet_fold=0_mixed_cnn'

now = datetime.now()
sub_file = 'submission_' + info + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '_postproc.csv'
sub_file = os.path.join('..', 'results', sub_file)
df.to_csv(sub_file, index=False)

In [48]:
df.head()

,image_name,Type_1,Type_2,Type_3
0,0.jpg,0.222587,0.95000,0.244191
1,1.jpg,0.032502,0.95000,0.281421
2,10.jpg,0.022474,0.95000,0.002031
3,100.jpg,0.175312,0.95000,0.054444
4,101.jpg,0.950000,0.41864,0.057392


Merge two submissions

In [1]:
import numpy as np
import pandas as pd

In [2]:
df1 = pd.read_csv("../results/submission_fold=0_mixed_cnn_2017-06-08-21-49.csv")
df2 = pd.read_csv("../results/submission_mean_cv=6_squeezenet_2017-05-25-23-30.csv")

In [8]:
df1.head()

,image_name,Type_1,Type_2,Type_3
0,0.jpg,0.359289,0.511135,0.129576
1,1.jpg,0.032502,0.686077,0.281421
2,10.jpg,0.022474,0.975495,0.002031
3,100.jpg,0.175312,0.770244,0.054444
4,101.jpg,0.523968,0.418640,0.057392


In [9]:
df2.head()

,image_name,Type_1,Type_2,Type_3
0,0.jpg,0.222587,0.533222,0.244191
1,1.jpg,0.215771,0.542684,0.241544
2,10.jpg,0.330304,0.534667,0.135029
3,100.jpg,0.300007,0.633719,0.066274
4,101.jpg,0.475317,0.412831,0.111853


In [69]:
def merge(df1, df2):
    types = ['Type_1', 'Type_2', 'Type_3']
    ll = min(df1.shape[0], df2.shape[0])
    df_out = pd.DataFrame(columns=df1.columns)
    for i in range(ll):    
        proba1 = df1.loc[i, types]
        proba2 = df2.loc[i, types]        
        proba = proba1 if np.max(proba1) > np.max(proba2) else proba2    
        df_out.loc[i, :] = (df1.loc[i, 'image_name'], ) + tuple(proba)  
    return df_out

In [40]:
df3 = merge(df1, df2)

In [41]:
df3.head()

,image_name,Type_1,Type_2,Type_3
0,0.jpg,0.222587,0.533222,0.244191
1,1.jpg,0.0325019,0.686077,0.281421
2,10.jpg,0.0224738,0.975495,0.00203107
3,100.jpg,0.175312,0.770244,0.0544437
4,101.jpg,0.523968,0.41864,0.0573924


In [42]:
import os
from datetime import datetime

info = 'merge__mean_cv=6_squeezenet_fold=0_mixed_cnn'

now = datetime.now()
sub_file = 'submission_' + info + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
sub_file = os.path.join('..', 'results', sub_file)
df3.to_csv(sub_file, index=False)

Compare dfs

In [154]:
def compute_mean(predictions):
    df = predictions[0]
    for p in predictions[1:]:
        df = pd.concat([df, p], axis=0)
    df = df.apply(pd.to_numeric, errors='ignore')
    gb = df.groupby('image_name')
    df2 = gb.agg(np.mean).reset_index()
    return df2

In [164]:
df_d1 = pd.read_csv("../../Intel_MobileODT_delivery/results/_submission_mean_cv=6_squeezenet.csv")
#df_d2 = pd.read_csv("../../Intel_MobileODT_delivery/results/_submission_mean_mixed_cnn.csv")
df_d21 = pd.read_csv("../../Intel_MobileODT_delivery/results/_submission_mixed_cnn_run_counter=1.csv")
df_d22 = pd.read_csv("../../Intel_MobileODT_delivery/results/_submission_mixed_cnn_run_counter=2.csv")
df_d2 = compute_mean([df_d21, df_d22])

df_t1 = pd.read_csv("../results/submission_mean_cv=6_squeezenet_2017-05-25-23-30.csv")
df_t2 = pd.read_csv("../results/submission_fold=0_mixed_cnn_2017-06-08-21-49.csv")

df_true = pd.read_csv("../results/stage1.csv")

In [179]:
df_final = pd.read_csv("../../Intel_MobileODT_delivery/results/submission_final_classification.csv")

In [165]:
df_m2 = pd.merge(df_d22, df_t2, left_index=True, right_index=True)

In [166]:
df_m2.head(10)

,image_name_x,Type_1_x,Type_2_x,Type_3_x,image_name_y,Type_1_y,Type_2_y,Type_3_y
0,0.jpg,0.283605,0.577240,0.139154,0.jpg,0.359289,0.511135,0.129576
1,1.jpg,0.030557,0.686122,0.283321,1.jpg,0.032502,0.686077,0.281421
2,10.jpg,0.023010,0.969214,0.007775,10.jpg,0.022474,0.975495,0.002031
3,100.jpg,0.369781,0.591814,0.038404,100.jpg,0.175312,0.770244,0.054444
4,101.jpg,0.262556,0.674967,0.062477,101.jpg,0.523968,0.418640,0.057392
5,102.jpg,0.172298,0.731564,0.096138,102.jpg,0.159052,0.769726,0.071222
6,103.jpg,0.128565,0.058112,0.813323,103.jpg,0.052909,0.023470,0.923622
7,104.jpg,0.049999,0.853241,0.096759,104.jpg,0.035460,0.882915,0.081624
8,105.jpg,0.058712,0.091677,0.849611,105.jpg,0.044025,0.101189,0.854786
9,106.jpg,0.025777,0.871318,0.102905,106.jpg,0.050637,0.827829,0.121534


In [167]:
df_true = df_true.sort_values('image_name')
df_true.head(10)

,image_name,Type_1,Type_2,Type_3
0,0.jpg,0.0,1.0,0.0
1,1.jpg,0.0,1.0,0.0
10,10.jpg,0.0,1.0,0.0
100,100.jpg,1.0,0.0,0.0
101,101.jpg,0.0,1.0,0.0
102,102.jpg,0.0,1.0,0.0
103,103.jpg,1.0,0.0,0.0
104,104.jpg,0.0,1.0,0.0
105,105.jpg,0.0,1.0,0.0
106,106.jpg,0.0,1.0,0.0


In [168]:
import os
import sys
# Project
project_common_path = os.path.dirname('.')
project_common_path = os.path.abspath(os.path.join(project_common_path, '..', 'common'))
if not project_common_path in sys.path:
    sys.path.append(project_common_path)

In [169]:
import numpy as np
from metrics import logloss_mc

In [170]:
types = ['Type_1', 'Type_2', 'Type_3']

In [171]:
df_true = df_true.sort_values('image_name')

In [172]:
df_true[types].values.astype(np.uint8)

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ..., 
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1]], dtype=uint8)

In [173]:
df5 = merge(df_d1, df_d2)

In [174]:
# Crop between [0.02, 0.98]
for t in ['Type_1', 'Type_2', 'Type_3']:
    mask = df5[t] < 0.03
    df5.loc[mask, t] = 0.03
    mask = df5[t] > 0.97
    df5.loc[mask, t] = 0.97

In [180]:
logloss_mc(df_true[types].values.astype(np.uint8), df5[types].values)

0.7877617354581079

In [51]:
logloss_mc??

In [178]:
import os
from datetime import datetime

info = 'merge_regularized__mean_cv=5_squeezenet+fold=0_mixed_cnn'

now = datetime.now()
sub_file = 'submission_' + info + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
sub_file = os.path.join('..', 'results', sub_file)
df5.to_csv(sub_file, index=False)

In [177]:
info = 'mean_mixed_cnn'
now = datetime.now()
sub_file = 'submission_' + info + '.csv'
sub_file = os.path.join('..', 'results', sub_file)
df_d2.to_csv(sub_file, index=False)

Play with logloss function

In [39]:
import os
import sys
# Project
project_common_path = os.path.dirname('.')
project_common_path = os.path.abspath(os.path.join(project_common_path, '..', 'common'))
if not project_common_path in sys.path:
    sys.path.append(project_common_path)

In [40]:
import numpy as np
from metrics import logloss_mc

In [17]:
n=100

In [18]:
y_true = np.zeros((n, 3), dtype=np.uint8)
y_true[:, 0] = 1

In [58]:
y_pred = 0.33 * np.ones((n, 3))
index = np.random.randint(0, 3, size=n)

In [59]:
logloss_mc(y_true, y_pred)

1.0986122886681107

In [95]:
y_pred = 0.2 * np.ones((n, 3))
index = np.random.randint(0, 3, size=n)
y_pred[np.arange(n), index] = 0.85

In [96]:
logloss_mc(y_true, y_pred)

1.3406290095499604